<a href="https://colab.research.google.com/github/ChrDobbert/web_scraping/blob/main/Crawler_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scrapy

In [ ]:
import time
from scrapy import Selector
# Import the CrawlerProcess
from scrapy.crawler import CrawlerProcess


from urllib.parse import urljoin
import scrapy
# Import requests to load text from URL
import requests

# CRAWLER

In [ ]:
# Create the Spider class
class MammutSpider(scrapy.Spider):
  name = "MammutSpider"
  
  # start_requests method
  def start_requests(self):  
    urls = [
            "https://www.mammut.com/ch/de/cat/1010/bekleidung/?filter.availability=Available&page=1"
    ]
    
    for url in urls:
      time.sleep(1)
      yield scrapy.Request(url = url, callback = self.parse_get_pages)
      
  def parse_get_pages(self, response):
    # Parser
    links = response.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "mm-product__product-url", " " ))]/@href').getall()
    for link in links:
        yield scrapy.Request(url = link, callback = self.parse_get_products)

  def parse_get_products(self, response):
    
    product_title = response.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "buy-box__title", " " ))]/text()').extract_first().strip()
    product_subtitle = response.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "buy-box__subtitle", " " ))]/text()').extract_first()
    product_prices = response.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "buy-box__price", " " ))]/text()').extract_first()
    product_purpose = response.css('.product-detail-page__purpose-item-icon').xpath('@alt').extract()
    product_description = response.css('.product-detail-page__description-text::text').extract()
    product_content = response.css('li.product-description__pointed-list-item').extract()
    print(product_title)
    # get product detail = material and selling points
    product_detail = response.css('.product-detail-page').getall()

    str_product_detail = str(product_detail)
    materialstr = str(product_detail)

    sellingPoints = re.findall(r'(?<=sellingPoints":).*$', str_product_detail)
    sellingPoints2 = re.findall('^(.+?)}', sellingPoints[0] )

    material = re.findall(r'(?<="Material &amp; Technologie","materials":).*$', materialstr)
    material2 = re.findall('^(.+?)}]', material[0] )

    mammut_dict[product_title] = [product_title,product_subtitle, product_prices, product_purpose, product_content, product_description, sellingPoints2, material2]

#_________________________________________________________________________   
# Initialize the dictionary **outside** of the Spider class
mammut_dict = dict()
#links = []

# Run the Spider
process = CrawlerProcess()
process.crawl(MammutSpider)
process.start()


# Print a preview of content
print(mammut_dict)



In [ ]:
print(mammut_dict)

{}


# Repository

In [ ]:

_____________________________________

# First parsing method
  def parse_front(self, response):
    course_blocks = response.css('div.course-block')
    course_links = course_blocks.xpath('./a/@href')
    links_to_follow = course_links.extract()
    for url in links_to_follow:
      yield response.follow(url = url,
                            callback = self.parse_pages)
# Second parsing method
  def parse_pages(self, response):
    
    # Create a SelectorList of the course titles text
    crs_title = response.xpath('//h1[contains(@class,"title")]/text()')
    
    # Extract the text and strip it clean
    crs_title_ext = crs_title.extract_first().strip()
    
    # Create a SelectorList of course descriptions text
    crs_descr = response.css( 'p.course__description::text' )
    
    # Extract the text and strip it clean
    crs_descr_ext = crs_descr.extract_first().strip()
   
    # Fill in the dictionary
    dc_dict[crs_title_ext] = crs_descr_ext

# CSV writer

In [ ]:
import csv

with open("crawler_output.csv", "w", newline="") as csvfile: 
  articlewriter = csv.writer(csvfile, delimiter=';', quotechar='"',quoting=csv.QUOTE_MINIMAL)

  for article in articles:
    articlewriter.writerow([article.emoji, article.title, article.image, article.content])

# SELECTOR

In [ ]:
# Import a scrapy Selector
from scrapy import Selector
from urllib.parse import urljoin
import time
# Import requests to load text from URL
import requests
import re


links = []

url_dict = {
  "men_url": "https://www.mammut.com/ch/de/cat/1010/bekleidung/?filter.availability=Available&page=%s",
  "women_url": "https://www.mammut.com/ch/de/cat/2010/bekleidung/?filter.availability=Available&page=%s"  
}

#url_dict.get("men_url")

def get_links(main_url):

  html = requests.get( main_url % 1 ).content

  # Create the Selector object sel from html
  sel = Selector( text = html )

  # Extract from selector
  next_page = sel.css('.mm-product-list-page__pagination').extract()
  str_x = str(next_page)
  next_page_2 = re.findall(r'(?<=init-pages-quantity=).*$', str_x)
  last_page = (int(next_page_2[0].split()[0].replace('"',"")))

  for i in range(1,last_page+1):
    time.sleep(1)
    url = main_url % i
    print(url)
    html = requests.get( url ).content
    sel = Selector( text = html )
    products = sel.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "mm-product__product-url", " " ))]/@href').getall()
    for i in products:
      links.append(urljoin(mammut,i))
  
get_links(url_dict.get("men_url"))  
get_links(url_dict.get("women_url"))

print(len(links))


In [ ]:
#print(links)

x = "https://www.mammut.com/ch/de/cat/1010/bekleidung/?filter.availability=Available&page=1"

if "2010" in x:
  gender = "female"
elif "1010" in x:
  gender = "male"

print(gender)

male


In [ ]:
import re
product_list = []
x = 0
product_list.append(["product_title", "product_subtitle", "gender", "product_price","product_price_old","sale_type", "product_description"])
urls = links
for url in urls:
  x = x + 1
  print(x)
  #time.sleep(1)
  if "women" in url:
    gender = "women"
  elif "men" in url:
    gender = "men"
  else:
    gender = "unisex"
  print(gender)
  
  html = requests.get( url ).content
  sel = Selector( text = html )
  product_title = sel.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "buy-box__title", " " ))]/text()').extract_first().strip()
  product_subtitle = sel.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "buy-box__subtitle", " " ))]/text()').extract_first()
  product_price = sel.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "buy-box__prices", " " ))]/div[1]/text()').extract_first().replace("CHF ","")
  product_price_old = sel.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "buy-box__prices", " " ))]/div[2]/text()').extract_first()
  if product_price_old == None:
    product_price_old = 0
  else:
    product_price_old = product_price_old.replace("CHF ","")
  sale_type = sel.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "product-badge product-badge--label product-detail-page__badge", " " ))]/text()').extract_first()
  print(sale_type)
  product_description = sel.css('.product-detail-page__description-text::text').extract()
  product_description = product_description[0]
  #product_content = sel.css('li.product-description__pointed-list-item').extract()
  # get product detail = material and selling points
  #product_detail = sel.css('.product-detail-page').getall()
  #str_product_detail = str(product_detail)
  #materialstr = str(product_detail)
  #sellingPoints = re.findall(r'(?<=sellingPoints":).*$', str_product_detail)
  #sellingPoints2 = re.findall('^(.+?)}', sellingPoints[0] )
  #material = re.findall(r'(?<="Material &amp; Technologie","materials":).*$', materialstr)
  #material2 = re.findall('^(.+?)}]', material[0] )
  #product_list.append([product_title,product_subtitle, product_prices, product_purpose, product_content, product_description, sellingPoints2, material2])
  product_list.append([product_title, product_subtitle, gender, product_price, product_price_old, sale_type, product_description])
#print(product_list)

In [ ]:
#print(product_list)
print(len(product_list))

1024


In [ ]:
print(product_list[0:5])

[['product_title', 'product_subtitle', 'gender', 'product_price', 'product_price_old', 'sale_type', 'product_description'], ['Sloper', 'T-Shirt für Herren', 'male', '55.00', 0, None, 'Sloper T-Shirt Men. Kletter-T-Shirt mit angesagtem Print. Das Oberteil kommt in einer weichen bioRe®-Baumwollqualität und wurde unter ökologischen Richtlinien sowie unter fairen Arbeitsbedingungen produziert.'], ['Convey Tour', 'Hardshell-Jacke mit Kapuze für Herren', 'male', '280.00', 0, None, 'Convey Tour HS Hooded Jacket Men. Die Regenjacke für absoluten Wetterschutz und mit allen Funktionen einer Wanderjacke. Wasserdicht. Winddicht. Atmungsaktiv. Dank GORE-TEX® Paclite. Der hochwertige und leichte Stoff macht die Hardshelljacke besonders bequem. Dank Easy Combine lässt sich die Jacke mit unterschiedlichen Isolations-Jacken und Midlayern kombinieren. Mehr Komfort. Mehr Leichtigkeit. Mehr Performance. Mit der Convey Tour HS Hooded Jacket Men.'], ['Zinal Guide', 'Hosen für Herren', 'male', '220.00', 0, N

# Create spreadsheet in google drive

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

#sh = gc.create('A new spreadsheet')

# Open our new sheet and add some data.
wks = gc.open('A new spreadsheet').sheet1

for entry in product_list[501:]:
  #print(entry)
  wks.append_row(entry)



In [ ]:
# get_all_values gives a list of rows.
rows = wks.get_all_values()
print(rows)